In [1]:
from collections import namedtuple
from enum import Enum

In [2]:
THRESHOLD = 1024

In [3]:
class LambdaType(Enum):
  func = 1
  appl = 2
  term = 3

In [4]:
isFunc = lambda expr: expr.getType() == LambdaType.func
isAppl = lambda expr: expr.getType() == LambdaType.appl
isTerm = lambda expr: expr.getType() == LambdaType.term

In [5]:
isParent = lambda expr: not isTerm(expr)

In [6]:
isFat = lambda expr: isFunc(expr) or isAppl(expr) or isTerm(expr)

In [7]:
brackets = lambda expr, fun, b: ('(['[b] + expr.toStr(1 - b) + ')]'[b]
    if fun(expr) else expr.toStr(b))

In [8]:
def lambdaCall(expr):
  seen = set()
  while True:
    seen.add(expr)
    assert len(seen) < THRESHOLD
    cur = expr.step()
    if not cur: return lambdaClean(expr)
    assert len(cur) < THRESHOLD and cur not in seen
    expr = cur

In [9]:
def lambdaClean(expr):
  while True:
    cur = expr.clean()
    if not cur: return expr
    expr = cur

In [10]:
class Func(namedtuple('Func', ['body'])):
  def __init__(self, *args, **kwargs):
    assert isFat(self.body)
  def __repr__(self):
    return self.toStr()
  def __len__(self):
    return len(self.body) + 1
  def __call__(self):
    return lambdaClean(self)
  def getType(self):
    return LambdaType.func
  def step(self):
    return None
  def clean(self):
    child = self.body.clean()
    return Func(child) if child else None
  def subst(self, arg, level):
    newBody, count = self.body.subst(arg.deepen(level=0), level + 1)
    return Func(newBody), count
  def deepen(self, level):
    return Func(self.body.deepen(level + 1))
  def toStr(self, b=0):
    return '♥' + brackets(self.body, isAppl, b)

In [11]:
class Appl(namedtuple('Appl', ['func', 'arg'])):
  def __init__(self, *args, **kwargs):
    assert isFat(self.func) and isFat(self.arg)
  def __repr__(self):
    return self.toStr()
  def __len__(self):
    return len(self.func) + len(self.arg) + 1
  def __call__(self):
    return lambdaCall(self)
  def getType(self):
    return LambdaType.appl
  def step(self):
    if isFunc(self.func): return self.func.body.subst(self.arg, level=0)[0]
    child = self.func.step()
    return Appl(child, self.arg) if child else None
  def clean(self):
    if isFunc(self.func):
      newBody, count = self.func.body.subst(self.arg, level=0)
      if count < 2: return newBody
    newFunc = self.func.clean()
    newArg = self.arg.clean()
    if not (newFunc or newArg): return None
    return Appl(newFunc or self.func, newArg or self.arg)
  def subst(self, arg, level):
    newFunc, funcCount = self.func.subst(arg, level)
    newArg, argCount = self.arg.subst(arg, level)
    return Appl(newFunc, newArg), funcCount + argCount
  def deepen(self, level):
    return Appl(self.func.deepen(level), self.arg.deepen(level))
  def toStr(self, b=0):
    return '{}.{}'.format(brackets(self.func, isFunc, b),
        brackets(self.arg, isParent, b))

In [12]:
class Term(namedtuple('Term', ['level'])):
  def __init__(self, *args, **kwargs):
    assert type(self.level) is int
  def __repr__(self):
    return self.toStr()
  def __len__(self):
    return 1
  def getType(self):
    return LambdaType.term
  def step(self):
    if self.level < 0: raise Exception('Error code: {}'.format(self.level))
    return None
  def clean(self):
    return None
  def subst(self, arg, level):
    if self.level == level: return arg, 0 if isTerm(arg) else 1
    return self if self.level < level else Term(self.level - 1), 0
  def deepen(self, level):
    return self if self.level < level else Term(self.level + 1) 
  def toStr(self, b=0):
    return '∞' if self.level < 0 else repr(self.level)

In [13]:
MakeTerm = lambda arg: Term(arg) if type(arg) is int else arg

In [14]:
L = lambda level, *args: Func(L(level - 1, *args) if level > 1 else
    C(*args) if len(args) > 1 else MakeTerm(args[0]))

In [15]:
C = lambda *args: (C(C(*args[:-1]), args[-1]) if len(args) > 2
    else Appl(MakeTerm(args[0]), MakeTerm(args[1])))

In [16]:
Id = L(1, 0)
Id

♥0

In [17]:
def PrintInts(pythonInt, num):
  assert type(pythonInt) is int
  assert isFunc(num)
  print('{:2} is {}'.format(pythonInt, C(num, Id)()))

In [18]:
Zero = L(1, 0, L(3, 0))
PrintInts(0, Zero)

 0 is ♥♥♥0


In [19]:
# Here's a function to increment an unbounded nonnegative integer in an efficient binary representation.
# TODO(tamer): Implement a compiler to generate lambda expressions like Inc and Dec from source code.

Inc = L(1, L(1, L(1, L(1, 0, 0, 3), L(2, 0,
    L(1, 2, 2, 0, 3), C(3, L(4, 2, 3)), C(2, 0))),
    C(0, L(4, 1, 3))), L(3, 0, C(2, 1)))

In [20]:
Inc

♥([♥([♥([♥(0.0.3)].[♥♥(0.[♥(2.2.0.3)].[3.(♥♥♥♥[2.3])].[2.0])])].[0.(♥♥♥♥[1.3])])].[♥♥♥(0.[2.1])])

In [21]:
num = Zero
PrintInts(0, Zero)
for i in range(16):
  num = C(num, Inc)()
  PrintInts(i + 1, num)
Sixteen = num

 0 is ♥♥♥0
 1 is ♥♥♥(1.[♥♥♥0])
 2 is ♥♥♥(2.[♥♥♥0])
 3 is ♥♥♥(1.[♥♥♥(1.[♥♥♥0])])
 4 is ♥♥♥(2.[♥♥♥(1.[♥♥♥0])])
 5 is ♥♥♥(1.[♥♥♥(2.[♥♥♥0])])
 6 is ♥♥♥(2.[♥♥♥(2.[♥♥♥0])])
 7 is ♥♥♥(1.[♥♥♥(1.[♥♥♥(1.[♥♥♥0])])])
 8 is ♥♥♥(2.[♥♥♥(1.[♥♥♥(1.[♥♥♥0])])])
 9 is ♥♥♥(1.[♥♥♥(2.[♥♥♥(1.[♥♥♥0])])])
10 is ♥♥♥(2.[♥♥♥(2.[♥♥♥(1.[♥♥♥0])])])
11 is ♥♥♥(1.[♥♥♥(1.[♥♥♥(2.[♥♥♥0])])])
12 is ♥♥♥(2.[♥♥♥(1.[♥♥♥(2.[♥♥♥0])])])
13 is ♥♥♥(1.[♥♥♥(2.[♥♥♥(2.[♥♥♥0])])])
14 is ♥♥♥(2.[♥♥♥(2.[♥♥♥(2.[♥♥♥0])])])
15 is ♥♥♥(1.[♥♥♥(1.[♥♥♥(1.[♥♥♥(1.[♥♥♥0])])])])
16 is ♥♥♥(2.[♥♥♥(1.[♥♥♥(1.[♥♥♥(1.[♥♥♥0])])])])


In [22]:
Dec = L(1, L(1, L(1, 0, 0, 2), L(2, 0, C(2, L(4, 1, 3)),
    L(1, L(1, 1, 0, 0, L(1, 0, 2)), L(1, 3, 3, 1,
    C(4, L(4, 2, 3)))), -1)), L(3, 0, C(2, 1)))

In [23]:
Dec

♥([♥([♥(0.0.2)].[♥♥(0.[2.(♥♥♥♥[1.3])].[♥([♥(1.0.0.[♥(0.2)])].[♥(3.3.1.[4.(♥♥♥♥[2.3])])])].∞)])].[♥♥♥(0.[2.1])])

In [24]:
num = Sixteen
PrintInts(16, num)
for i in range(16, 0, -1):
  num = C(num, Dec)()
  PrintInts(i - 1, num)
assert num == Zero

16 is ♥♥♥(2.[♥♥♥(1.[♥♥♥(1.[♥♥♥(1.[♥♥♥0])])])])
15 is ♥♥♥(1.[♥♥♥(1.[♥♥♥(1.[♥♥♥(1.[♥♥♥0])])])])
14 is ♥♥♥(2.[♥♥♥(2.[♥♥♥(2.[♥♥♥0])])])
13 is ♥♥♥(1.[♥♥♥(2.[♥♥♥(2.[♥♥♥0])])])
12 is ♥♥♥(2.[♥♥♥(1.[♥♥♥(2.[♥♥♥0])])])
11 is ♥♥♥(1.[♥♥♥(1.[♥♥♥(2.[♥♥♥0])])])
10 is ♥♥♥(2.[♥♥♥(2.[♥♥♥(1.[♥♥♥0])])])
 9 is ♥♥♥(1.[♥♥♥(2.[♥♥♥(1.[♥♥♥0])])])
 8 is ♥♥♥(2.[♥♥♥(1.[♥♥♥(1.[♥♥♥0])])])
 7 is ♥♥♥(1.[♥♥♥(1.[♥♥♥(1.[♥♥♥0])])])
 6 is ♥♥♥(2.[♥♥♥(2.[♥♥♥0])])
 5 is ♥♥♥(1.[♥♥♥(2.[♥♥♥0])])
 4 is ♥♥♥(2.[♥♥♥(1.[♥♥♥0])])
 3 is ♥♥♥(1.[♥♥♥(1.[♥♥♥0])])
 2 is ♥♥♥(2.[♥♥♥0])
 1 is ♥♥♥(1.[♥♥♥0])
 0 is ♥♥♥0
